# Wie hoch ist die Testabdeckung des Systems?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [ ]:
import py2neo
            
graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [ ]:
import pandas as pd

query ="MATCH (p:Jacoco:Package)-[HAS_CLASS]->(c:Jacoco:Class)-[:HAS_METHOD]->(m:Method:Jacoco)-[:HAS_COUNTER]->(t:Counter) WHERE t.type='INSTRUCTION' AND p.name STARTS WITH 'org' RETURN p.name AS package, ((sum(t.covered)*100)/(sum(t.covered)+sum(t.missed)))/100.0 as coverage, sum(t.covered)+sum(t.missed) as loc ORDER BY loc DESC"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [ ]:
df.head()

Die Daten werden analog zu Markus Harrers Blogbeitrag (https://feststelltaste.github.io/software-analytics/notebooks/vis/experimental/pandas-pygal/effective_charting.html) aufbereitet. Hierfür wird eine neue Spalte namens ```color``` im Dataframe erzeugt und mit einem Farbverlauf von grün nach rot entsprechend der Testabdeckung befüllt. Dafür wird die Colormap ```RdYlGn``` und die Methode ```rgb2hex()``` von matplotlib verwendet. 

In [ ]:
# Erzeuge eine neue Spalte color und berechne die entsprechende Farbe in Abhängigkeit zu coverage.
from matplotlib.cm import RdYlGn
from matplotlib.colors import rgb2hex

df['color'] = df['coverage'].apply(lambda x : rgb2hex(RdYlGn(x)))
df.head()


Die Testabdeckung für das gesamte System wird mittels der Methode ```mean()``` für die Spalte ```coverage``` berechnet und in der Variable ```coverage``` abgebildet.

In [ ]:
# Berechne den Mittelwert der Spalte coverage und speichere diesen in der Variable coverage.
coverage = df['coverage'].mean()
print(coverage)

## 4. Visualisierung
Die Daten werden mittels einer Treemap visualisiert (http://www.pygal.org/en/stable/documentation/types/treemap.html), wobei die Anzahl der Quelltextzeilen auf die Größe und die Testabdeckung auf die Farbe (rot = gering, grün = hoch) der Pakete abgebildet wird. Zudem wird die Testabdeckung für das gesamte System im Titel der Treemap angegeben.

In [ ]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [ ]:
# Erstelle Treemap.
import pygal
treemap = pygal.Treemap(height=250, show_legend=False, 
                        value_formatter = lambda x: 'Größe: {} Zeilen '.format(x))
treemap.title = 'Testabdeckung (Prozent bzw. grün=hoch & rot=gering) und Größe (Quelltextzeilen) auf Paketebene (gesamt: ' + str(coverage.round(2) * 100) + ' %)'
for index, row in df.iterrows():
    treemap.add(row['package'],
                [{"value": row['loc'], "label": "Testabdeckung: " + str(row['coverage'] * 100) + " %",
                  "color": row['color']}])
display(HTML(base_html.format(rendered_chart=treemap.render(is_unicode=True))))
